In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import xgboost as xgb

In [2]:
df = pd.read_csv("../input/1st-feature-selection-lasso-xgboost/new_df.csv")
df

,visit_year,visit_week,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,30,1,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,1,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,3,1,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,78,45,33,39,2,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,272,68,68,68,4,67,41,6,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,76,17,3,10,7,0,0,0,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,140,36,12,20,7,0,0,0,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,92,31,7,15,6,0,0,0,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,110,28,1,15,7,0,0,0,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [3]:
#new_df = df.drop(['sum_visitors'], axis=1)
#new_df

In [4]:
df_col = df.describe()
df_col = df_col.drop(['sum_visitors'], axis=1)
#(include ='int64')
df_col.columns

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude'],
      dtype='object')

In [5]:
varSel = pd.DataFrame({'Variable': df_col.columns})
varSel

,Variable
0,visit_year
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
6,sum_reserve_visitors
7,max_reserve_visitors
8,min_reserve_visitors
9,avg_reserve_visitors


In [6]:
nm = df_col.columns
nm = nm.append(pd.Index(['sum_visitors']))
nm

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude',
       'sum_visitors'],
      dtype='object')

In [7]:
df = df[nm].copy()
df.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors
0,2016,1,30,30,30,1,0,0,0,0,...,34.695124,135.197853,57,30,5,5,5,5,169.892977,30
1,2016,1,1,1,1,1,0,0,0,0,...,35.661777,139.704051,444,1,5,5,5,5,175.365828,1
2,2016,1,3,3,3,1,0,0,0,0,...,35.693840,139.703549,444,3,5,5,5,5,175.397390,3
3,2016,1,45,33,39,2,0,0,0,0,...,35.693840,139.703549,444,78,5,5,5,5,175.397390,78
4,2016,1,68,68,68,4,67,41,6,16,...,33.319286,130.508374,127,272,1,1,1,1,163.827660,272


In [8]:
X = df.iloc[:,:-1]
X.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,1,0,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,0,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,1,0,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,45,33,39,2,0,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,68,68,68,4,67,41,6,16,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660


In [9]:
y = df.iloc[:,-1:]
y.head()

,sum_visitors
0,30
1,1
2,3
3,78
4,272


In [10]:
### Variable Selection using SVM classification

In [11]:
svmmod = LinearSVC(C=0.01, penalty="l1",dual=False).fit(X, y.values.ravel())

In [12]:
model = SelectFromModel(svmmod, prefit=True)
model.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [13]:
varSel['SVM'] = model.get_support()
#.astype('int64')
varSel

,Variable,SVM
0,visit_year,True
1,visit_week,True
2,max_visitors,True
3,min_visitors,True
4,avg_visitors,True
5,count_visitors,True
6,sum_reserve_visitors,True
7,max_reserve_visitors,True
8,min_reserve_visitors,True
9,avg_reserve_visitors,True


### Summarization and Selection of Variables 


In [14]:
varSel['Sum'] = np.sum(varSel,axis=1)
varSel

,Variable,SVM,Sum
0,visit_year,True,1
1,visit_week,True,1
2,max_visitors,True,1
3,min_visitors,True,1
4,avg_visitors,True,1
5,count_visitors,True,1
6,sum_reserve_visitors,True,1
7,max_reserve_visitors,True,1
8,min_reserve_visitors,True,1
9,avg_reserve_visitors,True,1


In [15]:
varSel.groupby('Sum')['Variable'].count()

Sum
0     2
1    26
Name: Variable, dtype: int64

In [16]:
# We can now decide a threshold for selecting our variables!

In [17]:
varSel[varSel['Sum']>=1]

,Variable,SVM,Sum
0,visit_year,True,1
1,visit_week,True,1
2,max_visitors,True,1
3,min_visitors,True,1
4,avg_visitors,True,1
5,count_visitors,True,1
6,sum_reserve_visitors,True,1
7,max_reserve_visitors,True,1
8,min_reserve_visitors,True,1
9,avg_reserve_visitors,True,1


In [18]:
variables = varSel[varSel['Sum']>=1].iloc[:, [0]]
variables

,Variable
0,visit_year
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
6,sum_reserve_visitors
7,max_reserve_visitors
8,min_reserve_visitors
9,avg_reserve_visitors


In [19]:
variables_to_columns = variables.transpose().iloc[:].head()
df_variables_to_columns = pd.DataFrame(variables_to_columns)
sum_visitors = ['sum_visitors']
df_variables_to_columns['sum_visitors'] = sum_visitors
df_variables_to_columns.to_csv('second_variables_data.csv',index = False)
df_variables_to_columns

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,sum_visitors
Variable,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors


In [20]:
top_columns = pd.read_csv("./second_variables_data.csv",header = 1).columns
top_columns

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'number_of_holiday_days_visit', 'area_id',
       'genre_id', 'station_id', 'station_vincenty', 'station_great_circle',
       'restaurant_latitude', 'restaurant_longitude',
       'number_of_restaurants_in_area', 'weekend_visitors',
       'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude',
       'sum_visitors'],
      dtype='object')

In [21]:
df_After_Feature_Selection = pd.DataFrame(df[top_columns])
df_After_Feature_Selection.to_csv("./second_Feature_Selection.csv",index = False)
df_After_Feature_Selection

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors
0,2016,1,30,30,30,1,0,0,0,0,...,34.695124,135.197853,57,30,5,5,5,5,169.892977,30
1,2016,1,1,1,1,1,0,0,0,0,...,35.661777,139.704051,444,1,5,5,5,5,175.365828,1
2,2016,1,3,3,3,1,0,0,0,0,...,35.693840,139.703549,444,3,5,5,5,5,175.397390,3
3,2016,1,45,33,39,2,0,0,0,0,...,35.693840,139.703549,444,78,5,5,5,5,175.397390,78
4,2016,1,68,68,68,4,67,41,6,16,...,33.319286,130.508374,127,272,1,1,1,1,163.827660,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,17,3,10,7,0,0,0,0,...,35.659397,139.696440,444,152,5,5,5,5,175.355837,76
42277,2017,16,36,12,20,7,0,0,0,0,...,34.705362,135.510025,74,280,5,5,5,5,170.215387,140
42278,2017,16,31,7,15,6,0,0,0,0,...,35.661777,139.704051,444,184,5,5,5,5,175.365828,92
42279,2017,16,28,1,15,7,0,0,0,0,...,35.726232,139.723240,444,220,5,5,5,5,175.449472,110
